# Email process
## Ver 2.0
### 运行程序前将原始csv文件保存在当前目录的Email子目录，合并多个原始的csv格式email文件，删除空列，并按照原始邮件和回复邮件分别保存在不同Questions_data.csv文件和replys_data.csv中，提取对分类有帮助的Subject和Body列合并为Content列，保存在newdata.csv中。

In [1]:
import os
import pandas as pd

In [2]:
base_dir = "./email"
df = pd.DataFrame()
for item in os.listdir(base_dir):
    f = open(os.path.join(base_dir, item), encoding="utf-8")
    dftemp=pd.read_csv(f)

    df=pd.concat([df,dftemp])
#df.reset_index(inplace = True) 
#df.describe
#删除空列
df.drop(columns=['BCC: (Name)', 'BCC: (Address)','BCC: (Type)','Billing Information','Mileage'], inplace=True)
#删除Subject为空的行（只有1行不影响结果，第525行）
df.dropna(subset=['Subject'],inplace = True)
# 消除Subject首尾空格
df['Subject'] = df['Subject'].str.strip()
# 去掉多余的空格,步骤如下：
#   1.将字符串以空格拆分为列表，如"you are     man"拆分为["you","are","man"]
df['Subject']=df['Subject'].str.split()
#   2.将列表用空格连接，多余的空格被去掉
df['Subject']=df['Subject'].apply(' '.join)
# 同上
df['Body']=df['Body'].str.split()
df['Body']=df['Body'].apply(' '.join)
# 去掉多余的换行符，步骤如下：
#   1.将字符串以换行符(\n)拆分为列表，如"you\nare\n\n\nman"拆分为["you",,"are",,,"man"]
df['Body']=df['Body'].str.split('\n')
#   2.将列表中的空项删除
df['Body']=df['Body'].apply(lambda x:[t for t in x if len(t)>0])
#   3，将列表用换行符(\n)连接，多余的换行符被去掉
df['Body']=df['Body'].apply('\n'.join)

#print(df.info())
# 取出回复的邮件（邮件题目以RE:或FW:开始的邮件）
replysdf=pd.concat([df[df[df['Subject'].notnull()]['Subject'].str.startswith('RE:')]['Subject'],df[df[df['Subject'].notnull()]['Subject'].str.startswith('FW:')]['Subject']])
replysdf.to_csv(os.path.join(base_dir,"replys_data.csv"))
# 取出原始的提问邮件
questionsdf=df[~df.index.isin(replysdf.index)]
questionsdf.reset_index(inplace = True) 
questionsdf.to_csv(os.path.join(base_dir,"Questions_data.csv"))
print(questionsdf.info())
print(questionsdf)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            580 non-null    int64 
 1   Subject          580 non-null    object
 2   Body             580 non-null    object
 3   From: (Name)     580 non-null    object
 4   From: (Address)  580 non-null    object
 5   From: (Type)     580 non-null    object
 6   To: (Name)       564 non-null    object
 7   To: (Address)    564 non-null    object
 8   To: (Type)       564 non-null    object
 9   CC: (Name)       90 non-null     object
 10  CC: (Address)    90 non-null     object
 11  CC: (Type)       90 non-null     object
 12  Categories       253 non-null    object
 13  Importance       580 non-null    object
 14  Sensitivity      580 non-null    object
dtypes: int64(1), object(14)
memory usage: 68.1+ KB
None
     index                                            Subject  \
0       

In [3]:
# 查看其它列数据
cols=['From: (Name)','To: (Name)','CC: (Name)','Categories','Importance','Sensitivity']
for col in cols:
    group = questionsdf.groupby(col) # 按照'CC: (Name)'列的值来分组，创建一个groupby对象
    print('------ ',col, ' -----')
    for key, df in group:
        print(key,len(df))

------  From: (Name)  -----
Adriaan Nolst Trenité 2
Aldert de Vries 1
Alex-Sander Mulder 4
Alexandra Janssen 2
Alwin van Dijk 39
Angélique van Bladel-Dekkers 2
Anita Bakx-van der Voort 3
Ankie Benningshof 1
Anne-Sophie van der Hulst 1
Astrid Weijers 5
Astrid van Wingerden 1
Astrid van der Veer-Stuart 1
Bas Smits 1
Bauke van Sluis 4
Beleggingsadvies 2
Boudewijn Hasselman 1
Brent Atkins 1
Brigitte Cools-van Dinther 1
Brugt Jonker 1
CS Vraagbaak Postbus 1
Cassandra Smith-Cocx 6
Cees Weststrate 4
Centrale Orderdesk Postbus 1
Chris Zegers 1
Christiaan Minderhoud 1
Christine Van Bergen 1
ClientCenter 10
Coen van den Bedem 1
Cor Brinkers 1
Corinna from Phrase 1
Cypress.io 2
Daan van den Berg 2
Danielle van Leest 1
David Hanna 1
David Leguit 1
Eddy Goossen 2
Effecten Support Postbus 12
Eric Meiring 1
Eric Weusten 2
Erik Geurts 2
Esther Kok-Baars 2
Etiene van Braam 1
Ferdi Koch 2
Financieringscentrum Hypotheken Postbus 1
Folkert Jongepier 2
Francesco van Bekhoven 5
Frank Vervack 5
François Monc

### 暂时未发现数据对于分类有帮助，可暂时不考虑，以下仅处理Subject和Body，合并为Content列

In [4]:
# 合并'Subject','Body'列，生成newdata.csv文件
newdf=questionsdf[['Subject','Body']]
newdf=newdf.assign(Content=newdf['Subject'] + '\n' + newdf['Body'])
newdf.info()
newdf.to_csv(os.path.join(base_dir,"newdata.csv"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  580 non-null    object
 1   Body     580 non-null    object
 2   Content  580 non-null    object
dtypes: object(3)
memory usage: 13.7+ KB


In [5]:
import requests
import random
import json
import os
from hashlib import md5
import pandas as pd
from urllib import parse

In [6]:
base_dir='./email'
questionsfile='newdata.csv'
f = open(os.path.join(base_dir, questionsfile), encoding="utf-8")
questionsdf=pd.read_csv(f)
# 第0列为序号无意义，删除第0列unnamed
questionsdf.drop(questionsdf.columns[0],axis=1,inplace=True)
questionsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  580 non-null    object
 1   Body     576 non-null    object
 2   Content  580 non-null    object
dtypes: object(3)
memory usage: 13.7+ KB


In [7]:
# Set 百度翻译api的参数，appid/appkey等.
appid = '20200323000403030'
appkey = 'lvm1XikOF8TxxwA2B6j2'
endpoint = 'http://api.fanyi.baidu.com'
path = '/api/trans/vip/translate'
url = endpoint + path

In [8]:
# baidu translate function
def make_md5(s, encoding='utf-8'):
    return md5(s.encode(encoding)).hexdigest()
def translate_baidu(text, to_lang, from_lang='auto'):
    # Generate salt and sign
    text=parse.quote(text)
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + text + str(salt) + appkey)

    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': text, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result=r.json()
    # 百度免费文本翻译服务不允许高频调用，一次这里休眠5秒
    import time
    time.sleep(1)
 
    return parse.unquote(result['trans_result'][0]['dst'])

In [9]:
# 翻译的例子
text = newdf['Content'][1]
# 单次翻译文本长度限定为6000字节以内,每月的免费额度50000字符
print('source:',text)
# For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
to_lang =  'en'
result=translate_baidu(text,to_lang)
# 输出翻译结果
print('translate:',result)

source: BIS / Messaging Center (reactie verzocht)
Fijne collega! Ik zou graag het artikel Beleggen in de Tesla, Google, Amazon en Microsoft van de klimaattechnologie dat op de BIS willen doorsturen aan mijn relaties. Dit artikel kom ik echter niet tegen in het Messaging Center. Zie ik iets over het hoofd? Ik verneem graag. Met vriendelijke groet, Marco van Hofwegen Senior Beleggingsspecialist | Beleggingsadvies Team Regio Noord j.vanhofwegen@vanlanschot.com T +31 88 553 13 27 M +31 6 10 95 72 35 Apollolaan 150 1077 BG Amsterdam www.vanlanschot.nl Bauke van Sluis Relatiemanager | Beleggingsadvies Team Regio Noord b.vansluis@vanlanschot.com T +31 88 553 13 16 M +31 6 13 18 53 12 Van Lanschot introduceert interactieve app voor beleggingsadvies – Klik hier Van: Van Lanschot Beleggingsinformatie &lt;irc@vanlanschot.com&gt; Verzonden: donderdag 30 december 2021 12:00 Aan: Marco van Hofwegen &lt;J.vanHofwegen@vanlanschot.com&gt; Onderwerp: Over geld, beleggen en ons gedrag Het beste van Van L

In [10]:
# 对标题列进行翻译的示例，由于我是用的是免费额度，高频率调用会返回如下错误
# {'error_code': '54003', 'error_msg': 'Invalid Access Limit'}
# 这里的例子是翻译前三行
questionsdf=questionsdf[0:3]
# 对每行的Subject列进行翻译
questionsdf.loc[:,'Subject'] = questionsdf['Subject'].apply(lambda row: translate_baidu(row,to_lang))
print(questionsdf['Subject'])

0                     Name  does not show at query to 
1          BIS / Messaging Center (response requested)
2    A error  reported at portfolio SelectionObject...
Name: Subject, dtype: object


In [1]:
import os
import pandas as pd
from langchain.ll ms import AzureOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain 

In [2]:
base_dir='./email'
questionsfile='newdata.csv'
f = open(os.path.join(base_dir, questionsfile), encoding="utf-8")
questionsdf=pd.read_csv(f)
# 第0列为序号无意义，删除第0列unnamed
questionsdf.drop(questionsdf.columns[0],axis=1,inplace=True)
questionsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  580 non-null    object
 1   Body     576 non-null    object
 2   Content  580 non-null    object
 3   summary  579 non-null    object
dtypes: object(4)
memory usage: 18.3+ KB


In [3]:
# 初始化大模型，换成你自己的大模型调用
openai_api_key = "5d4d1d30e5704cc3b2812f8304e432fe" 

llm = AzureOpenAI(
    openai_api_key=openai_api_key, 
    azure_deployment="testchat",
    model_name="gpt-35-turbo",
    azure_endpoint="https://pubwebpoc.openai.azure.com/",
    openai_api_version="2023-05-15",
    temperature=0.7
#    streaming=True, 
#    callbacks=[StreamingStdOutCallbackHandler()]
    )
# 初始化拆分器
text_splitter = CharacterTextSplitter()

def get_summary(content):
    # 拆分文档
    texts = text_splitter.split_text(content)
    # 将拆分后的文本转成文档
    docs = [Document(page_content=t) for t in texts[:3]]
    # 生成摘要
    #load_summarize_chain的chain_type可以取值stuff、map_reduce、refine
    chain = load_summarize_chain(llm, chain_type="stuff")
    summary = chain.run(docs)
    
    return summary

questionsdf['summary'] = questionsdf['Content'].apply(get_summary)
    # 拆分文本
questionsdf.to_csv(os.path.join(base_dir,"newdata.csv"))
row=0
for row in questionsdf.shape[0]:
    print(questionsdf.iloc[row])
    row=row+1

C:\Users\UYUNWA\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `AzureOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureOpenAI instead.
  warn_deprecated(
C:\Users\UYUNWA\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


TypeError: 'int' object is not iterable

In [3]:
# 初始化大模型，换成你自己的大模型调用
openai_api_key = "5d4d1d30e5704cc3b2812f8304e432fe" 

llm = AzureOpenAI(
    openai_api_key=openai_api_key, 
    azure_deployment="testchat",
    model_name="gpt-35-turbo",
    azure_endpoint="https://pubwebpoc.openai.azure.com/",
    openai_api_version="2023-05-15",
    temperature=0.7
#    streaming=True, 
#    callbacks=[StreamingStdOutCallbackHandler()]
    )
# 初始化拆分器
text_splitter = CharacterTextSplitter()

def get_summary(content):
    # 拆分文档
    texts = text_splitter.split_text(content)
    # 将拆分后的文本转成文档
    docs = [Document(page_content=t) for t in texts[:3]]
    # 生成摘要
    #load_summarize_chain的chain_type可以取值stuff、map_reduce、refine
    prompt_template = """Write a concise summary of the following:

{text}

CONCISE SUMMARY IN ENGLISH:"""
    # 上面的prompt是要用英语语做摘要
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
    # summarize_chain
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
    summary = chain.run(docs)
 
    return summary

questionsdf['summary'] = questionsdf['Content'].apply(get_summary)
    # 拆分文本
questionsdf.to_csv(os.path.join(base_dir,"newdata.csv"))
questionsdf['summary']
i=0
for i in questionsdf['summary']:
    print(questionsdf['summary'][i])
    i=i+1

NameError: name 'AzureOpenAI' is not defined

  Using cached pip-23.3.2-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\UYUNWA\AppData\Local\anaconda3\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os 
import hdbscan
import pandas as pd
from langchain import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,) 

from dotenv import load_dotenv

In [3]:
base_dir='./email'
questionsfile='newdata.csv'
f = open(os.path.join(base_dir, questionsfile), encoding="utf-8")
df=pd.read_csv(f)
# 第0列为序号无意义，删除第0列unnamed
df.drop(df.columns[0],axis=1,inplace=True)
df.info()
#print(df['Content'][0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  580 non-null    object
 1   Body     576 non-null    object
 2   Content  580 non-null    object
 3   summary  579 non-null    object
dtypes: object(4)
memory usage: 18.3+ KB


In [6]:
pip install tiktoken

     -------------------------------------- 786.4/786.4 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
openai_api_key = "5d4d1d30e5704cc3b2812f8304e432fe" 
# 将Content列转换为Document类型
docs=[text for text in df['Content']]

embeddings = AzureOpenAIEmbeddings(
    openai_api_key=openai_api_key,
    azure_endpoint="https://pubwebpoc.openai.azure.com/",
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2023-07-01-preview",
    model = "text-embedding-ada-002"
    )
doc_result = embeddings.embed_documents(docs)
#embeddings = OpenAIEmbeddings(chunk_size=1000).embed_documents(docs)
#聚类分析
hdb = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3).fit(doc_result)
df['cluster']=hdb.labels_
# 删除不在群集中的文档
df = df.query("cluster != -1") 
print(df)

In [11]:
# 定义提示函数
def get_prompt():
    system_template = "You are an expert in summarization. You're going to help me write a clear title for the description."
    human_template = '''Using the following descriptions, write a title that summarizes them.
    
    QUESSTION DESCRIPTION:{questions_str}
    
    QUESSTION TITLE:'''
    
    return ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(system_template),
            HumanMessagePromptTemplate.from_template(human_template),
        ],
        input_variables=["questions"],
    )


llm=AzureChatOpenAI(
    openai_api_key=openai_api_key, 
    azure_endpoint="https://pubwebpoc.openai.azure.com/",
    deployment_name="testchat",
    model_name="gpt-35-turbo",
    openai_api_version="2023-07-01-preview",
    temperature=0.7, 
    )

for c in df.cluster.unique():
    chain = LLMChain(llm=llm, prompt=get_prompt(), verbose=False)
    questions_str = "\n".join(
        [
            f"{questions['Content']}\n"
            for questions in df.query(f"cluster == {c}").to_dict(orient="records")
        ]
    )
    result = chain.run({"questions": questions_str,})
    df.loc[df.cluster == c, "questions_title"] = result

ValueError: Missing some input keys: {'questions_str'}